In [ ]:
from pyimagesearch.transform import four_point_transform
from pyimagesearch import imutils
from skimage.filters import threshold_adaptive
import numpy as np
import cv2

In [ ]:
def DetectEdges(image):
    ratio = image.shape[0] / 500.0
    orig = image.copy()
    img = imutils.resize(image, height = 500)
    return img

In [ ]:
def FindEdges(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5,5), 0)
    edged = cv2.Canny(gray, 75, 200)
    return edged

In [ ]:
def FindContours(image):
    success = False
    
    (_,cnts,_) = cv2.findContours(image.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
    
    screenCnt = 0
    #loop over the contours 
    for c in cnts:
            #approximate the contour
            peri = cv2.arcLength(c,True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)
            
            # if our approximated contour has four points, then we
            # can assume that we have found our screen
            if len(approx) == 4:
                screenCnt = approx
                success = True
                break
                
    return screenCnt, success

In [ ]:
cap = cv2.VideoCapture(1)
count = 0

try:
    while(True):      
     
        # Capture frame-by-frame
        ret, frame = cap.read()
        
        if ret == False:
            continue
        
        if count%5 !=0:
            continue
        #Step 1: Edge Detection
#         image = DetectEdges(frame)
        edged = FindEdges(frame)
        
        cv2.imshow("Image", frame)
        cv2.imshow("Edged", edged)
        
        # STEP 2: Finding Contours
        screenCnt, success = FindContours(edged)
        if(success):
            cv2.drawContours(frame, [screenCnt], -1, (0, 255, 0), 2)
            cv2.imshow("Outline", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    cap.release()
    cv2.destroyAllWindows()
    print ("Released")
except Exception as e :
    cap.release()
    cv2.destroyAllWindows()
    print (e)    